In [3]:
# CÉLULA 1: Imports y Carga de Datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from scipy import sparse
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import time

# Imports de modelos
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import tensorflow as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print("🚀 Iniciando fase de entrenamiento...")

def load_processed_data(path='processed_data'):
    """Carga datos sparse y arrays numpy"""
    try:
        print(f"📂 Cargando datos desde '{path}'...")
        
        # Cargar matrices sparse
        X_train = sparse.load_npz(f'{path}/X_train_tfidf.npz')
        X_val = sparse.load_npz(f'{path}/X_val_tfidf.npz')
        X_test = sparse.load_npz(f'{path}/X_test_tfidf.npz')
        
        # Cargar etiquetas
        y_train = np.load(f'{path}/y_train.npy')
        y_val = np.load(f'{path}/y_val.npy')
        y_test = np.load(f'{path}/y_test.npy')
        
        print(f"✅ Datos cargados exitosamente.")
        print(f"   X_train shape: {X_train.shape} (Sparse)")
        print(f"   y_train shape: {y_train.shape}")
        return X_train, X_val, X_test, y_train, y_val, y_test
        
    except FileNotFoundError:
        print("❌ Error: No se encuentran los archivos. Asegúrate de haber ejecutado la fase de exportación.")
        return None

# Ejecutar carga
X_train, X_val, X_test, y_train, y_val, y_test = load_processed_data()

KeyError: '__reduce_cython__'

Modelos Clásicos (Naive Bayes y Regresión Logística)

In [ ]:
# CÉLULA 2: Modelos Clásicos (Baseline)

# Diccionario para guardar resultados
results = {}

def train_evaluate_model(model, name, X_tr, y_tr, X_v, y_v):
    """Función genérica para entrenar y evaluar modelos de sklearn"""
    print(f"\n🤖 Entrenando {name}...")
    start_time = time.time()
    
    # Entrenamiento
    model.fit(X_tr, y_tr)
    train_time = time.time() - start_time
    
    # Predicción
    y_pred = model.predict(X_v)
    
    # Métricas
    acc = accuracy_score(y_v, y_pred)
    f1 = f1_score(y_v, y_pred)
    
    print(f"✅ {name} completado en {train_time:.2f}s")
    print(f"   Accuracy (Val): {acc:.4f}")
    print(f"   F1-Score (Val): {f1:.4f}")
    
    return {
        'model': model,
        'accuracy': acc,
        'f1_score': f1,
        'time': train_time,
        'y_pred': y_pred
    }

# 1. Naive Bayes (MultinomialNB es el estándar para conteos de texto/TF-IDF)
nb_model = MultinomialNB()
results['Naive Bayes'] = train_evaluate_model(nb_model, 'Naive Bayes', X_train, y_train, X_val, y_val)

# 2. Logistic Regression
# class_weight='balanced' ayuda si hay desbalance leve, max_iter alto para asegurar convergencia
lr_model = LogisticRegression(C=1.0, solver='liblinear', random_state=42, max_iter=1000)
results['Logistic Regression'] = train_evaluate_model(lr_model, 'Logistic Regression', X_train, y_train, X_val, y_val)

Red Neuronal (Deep Learning)

In [ ]:
# CÉLULA 3: Red Neuronal (MLP)

print("\n🧠 Configurando Red Neuronal...")

# Las redes neuronales en Keras a veces prefieren matrices densas
# Si tienes mucha RAM (>16GB), puedes convertir todo. Si no, usa batches.
# Para 5000 features, convertir a denso suele ser seguro.
X_train_dense = X_train.toarray()
X_val_dense = X_val.toarray()

def build_neural_network(input_dim):
    model = Sequential([
        # Capa de entrada (5000 neuronas -> 64 neuronas)
        Dense(64, input_dim=input_dim, activation='relu'),
        Dropout(0.3), # Regularización para evitar Overfitting
        
        # Capa oculta
        Dense(32, activation='relu'),
        Dropout(0.2),
        
        # Capa de salida (1 neurona, sigmoide para binario 0/1)
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model_nn = build_neural_network(X_train.shape[1])
model_nn.summary()

# Callbacks para detener si no mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print("🔥 Entrenando Red Neuronal...")
start_time = time.time()

history = model_nn.fit(
    X_train_dense, y_train,
    epochs=20,           # Máximo de épocas
    batch_size=32,       # Ajustar según memoria
    validation_data=(X_val_dense, y_val),
    callbacks=[early_stopping],
    verbose=1
)

train_time = time.time() - start_time

# Evaluación NN
y_pred_prob = model_nn.predict(X_val_dense)
y_pred_nn = (y_pred_prob > 0.5).astype(int).flatten()

acc_nn = accuracy_score(y_val, y_pred_nn)
f1_nn = f1_score(y_val, y_pred_nn)

results['Neural Network'] = {
    'model': model_nn,
    'accuracy': acc_nn,
    'f1_score': f1_nn,
    'time': train_time,
    'history': history
}

print(f"✅ Neural Network completada en {train_time:.2f}s")
print(f"   Accuracy (Val): {acc_nn:.4f}")